# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

C. Gapp  ->  C. Gapp  |  ['C. Gapp']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
D. Christie  ->  D. Christie  |  ['D. Christie']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
T. Henning  ->  T. Henning  |  ['T. Henning']
G. Perotti  ->  G. Perotti  |  ['G. Perotti']
K. Schwarz  ->  K. Schwarz  |  ['K. Schwarz']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Liu  ->  J. Liu  |  ['J. Liu']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Paterson  ->  K. Paterson  |  ['K. Paterson']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
O. Krause  ->  O. Krause  |  ['O. Krause']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 80 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.02199


extracting tarball to tmp_2506.02199...

 done.


/tmp/ipykernel_3351/2822249172.py:52: LatexWarning: 2506.02199 did not run properly
bad escape \e at position 9
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[Anderson et~al.(2014)Anderson, Regan, Valenti, and Bergeron]{Anderson_2013}Rachel~E. Anderson, Michael Regan, Jeff Valenti, and Eddie Bergeron.\newblock Understanding persistence: A 3d trap map of an h2rg imaging sensor.\newblock \emph{arXiv e-prints}, art. arXiv:1402.4181, 2014.
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{Barbier} et~al.(2018){Barbier}, {Buton}, {Clemens}, {Conversi}, {Ealet}, {Ferriol}, {Fornari}, {Gillard}, {Kohley}, {Kubik}, {Rosset}, {Secroun}, {Serra}, {Smadja}, and {Zoubian}]{barbier18}R.~{Barbier}, C.~{Buton}, J.~C. {Clemens}, L.~{Co

extracting tarball to tmp_2506.03008...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/kfail.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width = 0.45\textwidth]{figures/kfail.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/US_v_SS_PMG_1.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=0.45\linewidth]{figures/US_v_SS_PMG_1.png}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcac

Found 127 bibliographic references in tmp_2506.03008/project2.bbl.
Error retrieving bib data for Blanchard:2019oqi: 'blanchard:2019oqi'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.02748-b31b1b.svg)](https://arxiv.org/abs/2506.02748) | **MINDS: The very low-mass star and brown dwarf sample. Detections and trends in the inner disk gas**  |
|| A. M. Arabhavi, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark> |
|*Appeared on*| *2025-06-04*|
|*Comments*| *20 pages, 18 figures, Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**|            Planet-forming disks around brown dwarfs and very low-mass stars (VLMS) are on average less massive and are expected to undergo faster radial solid transport than their higher mass counterparts. Spitzer had detected C$_2$H$_2$, CO$_2$ and HCN around these objects. With better sensitivity and spectral resolving power, JWST recently revealed incredibly carbon-rich spectra from such disks. A study of a larger sample of objects is necessary to understand how common such carbon-rich inner disk regions are and to put constraints on their evolution. We present and analyze MIRI observations of 10 disks around VLMS from the MIRI GTO program. This sample is diverse, with the central object ranging in mass from 0.02 to 0.14 $M_{\odot}$. They are located in three star-forming regions and a moving group (1-10 Myr). We identify molecular emission in all sources and report detection rates. We compare the molecular flux ratios between different species and to dust emission strengths. We also compare the flux ratios with the stellar and disk properties. The spectra of these VLMS disks are extremely molecular rich, and we detect the 10 $\mu$m silicate dust emission feature in 70% of the sample. We detect C$_2$H$_2$ and HCN in all of the sources and find larger hydrocarbons such as C$_4$H$_2$ and C$_6$H$_6$ in nearly all sources. Among O-bearing molecules, we find firm detections of CO$_2$, H$_2$O, and CO in 90%, 50%, and 20% of the sample, respectively. We find that the detection rates of organic molecules correlate with other organic molecules and anti-correlate with inorganic molecules. Hydrocarbon-rich sources show a weaker 10$\mu$m dust strength as well as lower disk dust mass than the oxygen-rich sources. We find potential evidence for C/O enhancement with disk age. The observed trends are consistent with models that suggest rapid inward solid material transport and grain growth.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.02775-b31b1b.svg)](https://arxiv.org/abs/2506.02775) | **Performance of the image persistence model for Euclid infrared detectors**  |
|| B. Kubik, et al. -- incl., <mark>K. Jahnke</mark>, <mark>K. Paterson</mark>, <mark>M. Schirmer</mark>, <mark>O. Krause</mark> |
|*Appeared on*| *2025-06-04*|
|*Comments*| *SPIE Astronomical Telescopes + Instrumentation, 2024, Yokohama, Japan*|
|**Abstract**|            Large-format infrared detectors are at the heart of major ground and space-based astronomical instruments, and the HgCdTe HxRG is the most widely used. The Near Infrared Spectrometer and Photometer (NISP) of the ESA's Euclid mission launched in July 2023 hosts 16 H2RG detectors in the focal plane. Their performance relies heavily on the effect of image persistence, which results in residual images that can remain in the detector for a long time contaminating any subsequent observations. Deriving a precise model of image persistence is challenging due to the sensitivity of this effect to observation history going back hours or even days. Nevertheless, persistence removal is a critical part of image processing because it limits the accuracy of the derived cosmological parameters. We will present the empirical model of image persistence derived from ground characterization data, adapted to the Euclid observation sequence and compared with the data obtained during the in-orbit calibrations of the satellite.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.03008-b31b1b.svg)](https://arxiv.org/abs/2506.03008) | **Euclid preparation. Constraining parameterised models of modifications of gravity with the spectroscopic and photometric primary probes**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-06-04*|
|*Comments*| *21 pages, 9 figures*|
|**Abstract**|            The Euclid mission has the potential to understand the fundamental physical nature of late-time cosmic acceleration and, as such, of deviations from the standard cosmological model, LCDM. In this paper, we focus on model-independent methods to modify the evolution of scalar perturbations at linear scales. We consider two approaches: the first is based on the two phenomenological modified gravity (PMG) parameters, $\mu_{\rm mg}$ and $\Sigma_{\rm mg}$, which are phenomenologically connected to the clustering of matter and weak lensing, respectively; and the second is the effective field theory (EFT) of dark energy and modified gravity, which we use to parameterise the braiding function, $\alpha_{\rm B}$, which defines the mixing between the metric and the dark energy field. We discuss the predictions from spectroscopic and photometric primary probes by Euclid on the cosmological parameters and a given set of additional parameters featuring the PMG and EFT models. We use the Fisher matrix method applied to spectroscopic galaxy clustering (GCsp), weak lensing (WL), photometric galaxy clustering (GCph), and cross-correlation (XC) between GCph and WL. For the modelling of photometric predictions on nonlinear scales, we use the halo model to cover two limits for the screening mechanism: the unscreened (US) case, for which the screening mechanism is not present; and the super-screened (SS) case, which assumes strong screening. We also assume scale cuts to account for our uncertainties in the modelling of nonlinear perturbation evolution. We choose a time-dependent form for $\{\mu_{\rm mg},\Sigma_{\rm mg}\}$, with two fiducial sets of values for the corresponding model parameters at the present time, $\{\bar{\mu}_0,\bar{\Sigma}_0\}$, and two forms for $\alpha_{\rm B}$, with one fiducial set of values for each of the model parameters, $\alpha_{\rm B,0}$ and $\{\alpha_{\rm B,0},m\}$. (Abridged)         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.02763-b31b1b.svg)](https://arxiv.org/abs/2506.02763) | **Homogeneous Stellar Atmospheric Parameters and 22 Elemental Abundances for FGK Stars Derived From LAMOST Low-resolution Spectra with DD-Payne**  |
|| M. Zhang, et al. -- incl., <mark>J. Shi</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2025-06-04*|
|*Comments*| *36 pages, 28 figures, 4 tables. Accepted for publication in ApJS*|
|**Abstract**|            A deep understanding of our Galaxy desires detailed decomposition of its stellar populations via their chemical fingerprints. This requires precise stellar abundances of many elements for a large number of stars. Here we present an updated catalog of stellar labels derived from LAMOST low-resolution spectra in a physics-sensible and rigorous manner with DD-Payne, taking labels from high-resolution spectroscopy as training set. The catalog contains atmospheric parameters for 6.4 million stars released in LAMOST DR9, and abundances for 22 elements, namely, C, N, O, Na, Mg, Al, Si, Ca, Ti, Cr, Mn, Fe, Ni, Sr, Y, Zr, Ba, La, Ce, Nd, Sm, and Eu, for nearly 3.6 million stars with spectral signal-to-noise ratio (SNR) higher than 20. The [Fe/H] is valid down to $\sim$-4.0, while elemental abundance ratios [X/Fe] are mostly valid for stars with [Fe/H] $\gtrsim-2.0$. Measurement errors in these labels are sensitive to and almost inversely proportional with SNR. For stars with S/N>50, we achieved a typical error of 30 K in Teff, 0.07 dex in $\log g$, $\sim0.05$ dex in abundances for most elements with atomic number smaller than Sr, and 0.1--0.2 dex for heavier elements. Homogenization to the label estimates is carried out via dedicated internal and external calibration. In particular, the non-local thermal equilibrium effect is corrected for the [Fe/H] estimates, the Teff is calibrated to the infrared flux method scale, and the $\log~g$ is validated with asteroseismic measurements. The elemental abundances are internally calibrated using wide binaries, eliminating systematic trend with effective temperature. The catalog is publicly available.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.02745-b31b1b.svg)](https://arxiv.org/abs/2506.02745) | **Euclid: Early Release Observations -- The surface brightness and colour profiles of the far outskirts of galaxies in the Perseus cluster**  |
|| M. Mondelin, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-06-04*|
|*Comments*| *29 pages, 20 figures*|
|**Abstract**|            The Perseus field captured by Euclid as part of its Early Release Observations provides a unique opportunity to study cluster environment ranging from outskirts to dense regions. Leveraging unprecedented optical and near-infrared depths, we investigate the stellar structure of massive disc galaxies in this field. This study focuses on outer disc profiles, including simple exponential (Type I), down- (Type II) and up-bending break (Type III) profiles, and their associated colour gradients, to trace late assembly processes across various environments. Type II profiles, though relatively rare in high dense environments, appear stabilised by internal mechanisms like bars and resonances, even within dense cluster cores. Simulations suggest that in dense environments, Type II profiles tend to evolve into Type I profiles over time. Type III profiles often exhibit small colour gradients beyond the break, hinting at older stellar populations, potentially due to radial migration or accretion events. We analyse correlations between galaxy mass, morphology, and profile types. Mass distributions show weak trends of decreasing mass from the centre to the outskirts of the Perseus cluster. Type III profiles become more prevalent, while Type I profiles decrease in lower-mass galaxies with cluster centric distance. Type I profiles dominate in spiral galaxies, while Type III profiles are more common in S0 galaxies. Type II profiles are consistently observed across all morphological types. While the limited sample size restricts statistical power, our findings shed light on the mechanisms shaping galaxy profiles in cluster environments. Future work should extend observations to the cluster outskirts to enhance statistical significance. Additionally, 3D velocity maps are needed to achieve a non-projected view of galaxy positions, offering deeper insights into spatial distribution and dynamics.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.02199-b31b1b.svg)](https://arxiv.org/abs/2506.02199) | **WASP-121b's transmission spectrum observed with JWST/NIRSpec G395H reveals thermal dissociation and SiO in the atmosphere**  |
|| <mark>C. Gapp</mark>, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>D. Christie</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-06-04*|
|*Comments*| *26 pages, 15 figures, 6 tables*|
|**Abstract**|            WASP-121b has been established as a benchmark ultrahot Jupiter, serving as a laboratory for the atmospheric chemistry and dynamics of strongly irradiated extrasolar gas giants. Here, we present and analyze WASP-121b's transmission spectrum observed with NIRSpec G395H on board the James Webb Space Telescope and find evidence for the thermal dissociation of H$_2$O and H$_2$ on the planet's permanent dayside. Additionally, we detect SiO at a statistical significance of $5.2\sigma$ which is compatible with chemical equilibrium in the atmosphere. Constraining the abundance of SiO and abundance ratios between silicon and volatile atoms in WASP-121b's atmosphere could help discriminate between possible migration histories of the planet. The three-dimensional nature of thermal dissociation on WASP-121b's dayside and of recombination on its nightside, however, poses a challenge to constraining molecular abundances and elemental abundance ratios from the transmission spectrum. To account for this, we implemented an atmospheric model in the NEMESIS framework that splits the planet's atmosphere into dayside and nightside. A retrieval applying our atmospheric model to WASP-121b's transmission spectrum favors a higher H$_2$O abundance on the nightside than on the dayside, demonstrating the impact of hemispheric heterogeneity when attempting to constrain WASP-121b's bulk H$_2$O inventory.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \e at position 9</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.02748/./dereddened_VLMS_features_PPVII_TWA27_1.0.3.png', 'tmp_2506.02748/./dereddened_correlations_PPVII_TWA27_v8.png', '', '', '', '']
copying  tmp_2506.02748/./dereddened_VLMS_features_PPVII_TWA27_1.0.3.png to _build/html/
copying  tmp_2506.02748/./dereddened_correlations_PPVII_TWA27_v8.png to _build/html/
file not found 
file not found 
file not found 
file not found 
exported in  _build/html/2506.02748.md
    + _build/html/tmp_2506.02748/./dereddened_VLMS_features_PPVII_TWA27_1.0.3.png
    + _build/html/tmp_2506.02748/./dereddened_correlations_PPVII_TWA27_v8.png
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
found figures ['tmp_2506.02775/././Figures/median_ground_alpha_evolution_with_sat_and_flight_sca_18628.png', 'tmp_2506.02775/././Figures/median_ground_beta_evolution_with_sat_and_flight_sca_18628.png', 'tmp_2506.02775/././Figures/median_signal_evolution_and_model_sca18267_irL_11_v2.png', 'tmp_2506.02775/././Figures/median_pe

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ama}[1]{\textcolor{red}{\rm #1^{(\rm AMA)}}}$
$\newcommand{\ik}[1]{\textcolor{purple}{\rm #1^{(\rm IK)}}}$
$\newcommand{\jay}[1]{\textcolor{magenta}{\rm #1^{(\rm Jay)}}}$
$\newcommand{\ch}[1]{{\rm #1}}$
$\newcommand{\hl}[1]{{{#1}}}$
$\newcommand{\hll}[1]{{{#1}}}$
$\newcommand{\PreserveBackslash}[1]{\let\temp=\\#1\let\\=\temp}$
$\newcommand{\mybox}[2][1.1ex]{\raisebox{#1}{\myboxi{#2}}}$
$\newcommand{\myboxx}[2][1.1ex]{\raisebox{#1}{\myboxxi{#2}}}$
$\newcommand{\myboxxx}[2][1.1ex]{\raisebox{#1}{\myboxxxi{#2}}}$</div>



<div id="title">

# MINDS: The very low-mass star and brown dwarf sample: Detections and trends in the inner disk gas

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.02748-b31b1b.svg)](https://arxiv.org/abs/2506.02748)<mark>Appeared on: 2025-06-04</mark> -  _20 pages, 18 figures, Accepted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

A. M. Arabhavi, et al. -- incl., <mark>T. Henning</mark>, <mark>G. Perotti</mark>, <mark>K. Schwarz</mark>

</div>
<div id="abstract">

**Abstract:** Planet-forming disks around brown dwarfs and very low-mass stars (VLMS) are on average less massive and are expected to undergo faster radial solid transport than their higher mass counterparts. _Spitzer_ had detected $\ch{C_2H_2}$ , $\ch{CO_2}$ and $\ch{HCN}$ around these objects, but did not provide a firm detection of water. With better sensitivity and spectral resolving power than that of _Spitzer_ , the _James Webb Space Telescope_ (JWST) has recently revealed incredibly carbon-rich spectra and only one water-rich spectrum from such disks. A study of a larger sample of objects is necessary to understand how common such carbon-rich inner disk regions are and to put constraints on their evolution. We present and analyze JWST MIRI/MRS observations of 10 disks around VLMS from the MIRI GTO program. This sample is diverse, with the central object ranging in mass from 0.02 to 0.14 $M_{\odot}$ . They are located in three star-forming regions and a moving group (1-10 Myr). We identify molecular emission $\hl{in all sources based on recent literature and spectral inspection,}$ and report detection rates. We compare the molecular flux ratios between different species and to dust emission strengths. We also compare the flux ratios with the stellar and disk properties. The spectra of these VLMS disks are extremely rich in molecular emission, and we detect the 10 $\mu$ m silicate dust emission feature in 70 \% of the sample. We detect $\ch{C_2H_2}$ and $\ch{HCN}$ in all of the sources and find larger hydrocarbons such as $\ch{C_4H_2}$ and $\ch{C_6H_6}$ in nearly all sources. Among oxygen-bearing molecules, we find firm detections of $\ch{CO_2}$ , $\ch{H_2O}$ , and $\ch{CO}$ in 90 \% , 50 \% , and 20 \% of the sample, respectively. We find that the detection rates of organic molecules correlate with other organic molecules and anti-correlate with the detection rates of inorganic molecules. Hydrocarbon-rich sources show a weaker 10 $\mu$ m dust strength as well as lower disk dust mass (measured from millimeter fluxes) than the oxygen-rich sources. We find potential evidence for C/O enhancement with disk age. The observed trends are consistent with models that suggest rapid inward solid material transport and grain growth.

</div>

<div id="div_fig1">

<img src="tmp_2506.02748/./dereddened_VLMS_features_PPVII_TWA27_1.0.3.png" alt="Fig17" width="100%"/>

**Figure 17. -** Summary of dust and gas observations. The left panels show continuum-normalized dust strengths highlighting $\ch${SiO_2}, enstatite, and forsterite emission features. The middle panels show a zoom in on the observed $\ch${C_2H_2} features highlighting four peaks corresponding to the main and rare isotopologues in red and blue respectively. The right panels show the continuum-subtracted hydrocarbon-rich wavelength region of the spectra. The grey regions highlight the main $Q$-branches of molecules in this wavelength region. $\ch${HCN^*} and $\ch${CO_2^*} indicate hot bands $\nu_2$:1-0 and $\nu_1\nu_2\nu_3$:100-010, respectively. The spectra are ordered by increasing flux ratios of the $\ch${^{13}CCH_2} and $\ch${C_2H_2} peaks (see Sect. \ref{sec:trends2}) and are normalised to a distance of 150 pc. (*fig:features_summary*)

</div>
<div id="div_fig2">

<img src="tmp_2506.02748/./dereddened_correlations_PPVII_TWA27_v8.png" alt="Fig23" width="100%"/>

**Figure 23. -** Binary correlation coefficients  (Yule1912)  between the detection rates of different molecules. The box on the bottom left describes the details of various numbers shown in each colored box on the right. The color denotes the value of the correlation coefficient, with red corresponding to an anti-correlation and blue corresponding to a correlation. For calculating the coefficients only firm detections are considered as `detected' while the rest (including tentative detections) are considered `not detected'. Molecules that are detected in all the sources or are detected in only one source are not shown. Molecule pairs with statistically significant correlations (p-value<0.05) are highlighted with black borders. (*fig:correlations*)

</div>
<div id="div_fig3">

<img src="" alt="Fig5.1" width="25%"/><img src="" alt="Fig5.2" width="25%"/><img src="" alt="Fig5.3" width="25%"/><img src="" alt="Fig5.4" width="25%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Shown in greyscale is a...Plotted above...Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.02748"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\baselinestretch}{1.0}$
$\newcommand{\thesubfigure}{\Alph{subfigure}}$</div>



<div id="title">

# Performance of the image persistence model for ${\LARGE_**Euclid**_}$ infrared detectors

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.02775-b31b1b.svg)](https://arxiv.org/abs/2506.02775)<mark>Appeared on: 2025-06-04</mark> -  _SPIE Astronomical Telescopes + Instrumentation, 2024, Yokohama, Japan_

</div>
<div id="authors">

B. Kubik, et al. -- incl., <mark>K. Jahnke</mark>, <mark>K. Paterson</mark>, <mark>M. Schirmer</mark>, <mark>O. Krause</mark>

</div>
<div id="abstract">

**Abstract:** Large-format infrared detectors are at the heart of major ground and space-based astronomical instruments, and the HgCdTe HxRG is the most widely used. The Near Infrared Spectrometer and Photometer (NISP) of the ESA's $\Euclid$ mission launched in July 2023 hosts 16 H2RG detectors in the focal plane. Their performance relies heavily on the effect of image persistence, which results in residual images that can remain in the detector for a long time contaminating any subsequent observations.Deriving a precise model of image persistence is challenging due to the sensitivity of this effect to observation history going back hours or even days. Nevertheless, persistence removal is a critical part of image processing because it limits the accuracy of the derived cosmological parameters.We will present the empirical model of image persistence derived from ground characterization data, adapted to the $\Euclid$ observation sequence and compared with the data obtained during the in-orbit calibrations of the satellite.

</div>

<div id="div_fig1">

<img src="tmp_2506.02775/././Figures/median_ground_alpha_evolution_with_sat_and_flight_sca_18628.png" alt="Fig4.1" width="50%"/><img src="tmp_2506.02775/././Figures/median_ground_beta_evolution_with_sat_and_flight_sca_18628.png" alt="Fig4.2" width="50%"/>

**Figure 4. -**  Median parameters $\alpha$(left panel) and $\beta$(right panel) of the persistence model for one detector as function of fluence. The unfilled circles show the parameters $\alpha_\text{gd}$ and $\beta_\text{gd}$ measured during ground characterizations for unsaturating stimuli. The * markers show the parameters $\alpha_\text{gd}$ and $\beta_\text{gd}$ from ground characterizations for saturating stimuli. The filled circular markers represent the coefficients measured from flight calibrations $\alpha_\text{flight}$ and $\beta_\text{flight}$. (*fig:alpha_beta_median*)

</div>
<div id="div_fig2">

<img src="tmp_2506.02775/././Figures/median_signal_evolution_and_model_sca18267_irL_11_v2.png" alt="Fig8.1" width="50%"/><img src="tmp_2506.02775/././Figures/median_persistence_evolution_and_model_sca18267_irL_11_v2.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Stabilisation of detector median stimulus amplitude (left panel) and persistence amplitude (right panel) in a sequence of measurements consisting of 15 pairs of flat and dark exposures at a constant nominal fluence, taken after the detector has been in the dark for a long period of time. The steady-state values of signal $S_\infty$ and persistence $P_\infty$ amplitudes are indicated by grey dashed lines. (*fig:pers_stabilisation*)

</div>
<div id="div_fig3">

<img src="tmp_2506.02775/././Figures/persistence_ratio_s_irL05_irL90_irL50_spectro_irL70_spectro.png" alt="Fig9" width="100%"/>

**Figure 9. -** Relative persistence in NISP detectors for typical photometric exposure of 87s. Unsaturating stimuli are represented by blue circles and green squares. Saturating fluences are represented by red triangles and grey pentagons. (*fig:pers_to_signal*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.02775"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\orcid}[1]$
$\newcommand{\bfpsi}{\bm\psi}$
$\newcommand{\bfell}{\bm\ell}$
$\newcommand{\bfn}{\bm n}$
$\newcommand{\bfth}{\bm\theta}$
$\newcommand{\bfz}{\bm z}$
$\newcommand{\bfs}{\bm s}$
$\newcommand{\bfv}{\bm v}$
$\newcommand{\bfx}{\bm x}$
$\newcommand{\bfk}{\bm k}$
$\newcommand{\bfp}{\bm p}$
$\newcommand{\bfq}{\bm q}$
$\newcommand{\bfr}{\bm r}$
$\newcommand{\bfu}{\bm u}$
$\newcommand{\bfw}{\bm w}$
$\newcommand{\bfy}{\bm y}$
$\newcommand{\bfPhi}{\bm\Phi}$
$\newcommand{\bfM}{\bm M}$
$\newcommand{\bfF}{\bm F}$
$\newcommand{\fnl}{f_{\rm NL}}$
$\newcommand{\gnl}{g_{\rm NL}}$
$\newcommand{\tdu}{\widetilde{u}}$
$\newcommand{\tdPhi}{\widetilde{\Phi}}$
$\newcommand{\tdG}{\widetilde{G}}$
$\newcommand{\tdP}{\widetilde{P}}$
$\newcommand{\tdR}{\widetilde{R}}$
$\newcommand{\bfkk}{\mbox{\boldmathk}}$
$\newcommand{\GCsp}{\text{GC}\ensuremath{_\mathrm{sp}}}$
$\newcommand{\GCph}{\text{GC}\ensuremath{_\mathrm{ph}}}$
$\newcommand{\Omegam}{\ensuremath{\Omega_{\mathrm{m},0}}}$
$\newcommand{\Omegab}{\ensuremath{\Omega_{\mathrm{b},0}}}$
$\newcommand{\OmegaDE}{\ensuremath{\Omega_{\mathrm{DE},0}}}$
$\newcommand{\ns}{n_{\mathrm{s}}}$
$\newcommand{\lcdm}{\ensuremath{\Lambda\mathrm{CDM}}}$
$\newcommand{\logfr}{\ensuremath{\log_{10}(|f_{R0}|)}}$
$\newcommand{\fr}{\ensuremath{|f_{R0}|}}$
$\newcommand{\sg}{\ensuremath{\sigma_{8}}}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\etal}{\emph{et~al}}$
$\newcommand{\fRpaper}[1]{{\color{orange}{#1}}}$
$\newcommand{\ISTcite}{Blanchard:2019oqi }$</div>



<div id="title">

# $\Euclid$ preparation: Constraining parameterised models of modifications of gravity with the spectroscopic and photometric primary probes

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.03008-b31b1b.svg)](https://arxiv.org/abs/2506.03008)<mark>Appeared on: 2025-06-04</mark> -  _21 pages, 9 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ mission has the potential to understand the fundamental physical nature of late-time cosmic acceleration and, as such, of deviations from the standard cosmological model, $\lcdm$ .In this paper, we focus on model-independent methods to modify the evolution of scalar perturbations at linear scales.We consider two powerful and convenient approaches: the first is based on the two phenomenological modified gravity (PMG) parameters, $\mu_{\rm mg}$ and $\Sigma_{\rm mg}$ , which are phenomenologically connected to the clustering of matter and weak lensing, respectively; and the second is the effective field theory (EFT) of dark energy and modified gravity, which we use to parameterise the braiding function, $\alpha_{\rm B}$ , which defines the mixing between the metric and the dark energy field typical of Galileon theories. We discuss the predictions from spectroscopic and photometric primary probes by $\Euclid$ on the cosmological parameters and a given set of additional parameters featuring the PMG and EFT models. We use the Fisher matrix method applied to spectroscopic galaxy clustering ( $\GCsp$ ), weak lensing (WL), photometric galaxy clustering ( $\GCph$ ), and cross-correlation (XC) between $\GCph$ and WL. For the modelling of photometric predictions on nonlinear scales, we use the halo model reaction approach to cover two limiting cases for the screening mechanism: the unscreened (US) case, for which the screening mechanism is not present; and the super-screened (SS) case, which assumes strong screening.We also assume scale cuts to account for our uncertainties in the modelling of nonlinear perturbation evolution. We choose a time-dependent form for $\{\mu_{\rm mg},\Sigma_{\rm mg}\}$ , with two fiducial sets of values for the corresponding model parameters at the present time, $\{\bar{\mu}_0,\bar{\Sigma}_0\}$ , and two forms for $\alpha_{\rm B}$ , with one fiducial set of values for each of the model parameters, $\alpha_{\rm B,0}$ and $\{\alpha_{\rm B,0},m\}$ . At the 68.3 \% confidence level, the percentage relative errors obtained with $\Euclid$ alone and our conservative settings for the full combination of probes for the US case are: for $\{\bar{\mu}_0,\bar{\Sigma}_0\}$ , with a $\lcdm$ fiducial, \{ $23.3\%$ , $2.6\%$ \} ; for a fiducial $\{\bar{\mu}_0,\bar{\Sigma}_0\}$ = $\{0.5,0.5\}$ we obtain ${$36.2\%$, $2.7\%$}$ ; for $\alpha_{\rm B,0}$ whose fiducial is $0.2$ , we have $31.1\%$ and for $\{\alpha_{\rm B,0},m\}$ with fiducial $\{0.9,2.4\}$ we have $11.6\%$ and $11.8\%$ . The constraints we obtain with the SS prescription provide similar values. We also compute the constraints for different combinations of probes to assess their standalone and complementary constraining power.

</div>

<div id="div_fig1">

<img src="" alt="Fig4.1" width="50%"/><img src="" alt="Fig4.2" width="50%"/>

**Figure 4. -** 68\% and 95\% joint marginal error contours on the cosmological parameters in the phenomenological modified gravity approach for the PMG-2 case, using different probe combinations: $\GCsp$(orange); 3\texttimes2 pt. statistics (blue); and $\GCsp$+3\texttimes2 pt. statistics (green). Two different nonlinear corrections have also been considered for the photometric probes: US (left panels); and SS (right panels). (*fig:ellipses-musigma_GC2*)

</div>
<div id="div_fig2">

<img src="" alt="Fig6.1" width="50%"/><img src="" alt="Fig6.2" width="50%"/>

**Figure 6. -** 3\texttimes2 pt. statistics 68\% marginal error on $\bar{\mu}_0$(left panel), $\bar{\Sigma}_0$(right panel) as a function of the $k^{\rm 3\times2pt}_{\rm max}$ value considered in the photometric analysis for the PMG-1 model when all other parameters are fixed to their fiducial values. (*fig:musigma_maxed*)

</div>
<div id="div_fig3">

<img src="" alt="Fig3.1" width="50%"/><img src="" alt="Fig3.2" width="50%"/>

**Figure 3. -** 68\% and 95\% joint marginal error contours on the cosmological parameters in the phenomenological modified gravity approach for the PMG-1 case, using different probe combinations: $\GCsp$(orange); 3\texttimes2 pt. statistics (blue); and $\GCsp$+3\texttimes2 pt. statistics (green). Two different nonlinear corrections have also been considered for the photometric probes: US (left panels); and SS (right panels).
  (*fig:ellipses-musigma_GC1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.03008"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

510  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
